# Introduction to FermiLib
### Version 0.1

## The FermionTerm data structure represents local fermionic operators

Fermionic systems are often treated in second quantization where arbitrary operators can be expressed using the fermionic creation and annihilation operators, $a^\dagger_k$ and $a_k$.  The fermionic ladder operators play a similar role to their qubit ladder operator counterparts, $\sigma^+_k$ and $\sigma^-_k$ but are distinguished by the cannonical fermionic anticommutation relations, $\{a^\dagger_i, a^\dagger_j\} = \{a_i, a_j\} = 0$ and $\{a_i, a_j^\dagger\} = \delta_{ij}$, also known as the Pauli exclusion principle. A "local fermionic operator" is defined here as a product of fermionic creation and annihilation operators with an associated (complex) scalar coefficient. For instance, $-1.7 \, a^\dagger_3 a_1$, $(1 + 2i) \, a^\dagger_4 a^\dagger_3 a_9 a_1$ and $0.8\, a_3 a_{11}$ are all perfectly valid examples of local fermionic operators.

Representation of these operators using the FermionTerm data structure is at the very heart of FermiLib. The FermionTerm class is part of the fermion_operators.py module. Because FermionTerms share the same basic structure as QubitTerms (which we introduce later on in this demo), both classes inherit most of their functionality from the LocalTerm parent class which is found in the module local_terms.py. FermionTerm objects store three important attributes: the number of sites on which the term acts (n_qubits), the scalar coefficient of the term (coefficient) and a python list which specifies the operators in the term (terms). The terms list is a list of 2-tuples which represent the $a^\dagger_k$ / $a_k$ ladder operators with the first element is an int giving the site on which the term acts and the second element is Boole indicating whether the operator is $a^\dagger$ (1), or $a$ (0). For instance, $[(3, 1), (7, 0)]$ represents $a_3^\dagger a_7$. Note that the "terms" attribute associated with the identity operator is an empty list.

In the cell below, we import FermionTerm and initialize a few example terms. We show that terms can be manipulated in an intuitive way using python operators such as $*=$, $==$, $!=$, len, abs. The slice, get, set and iteration methods are redirected to the .terms attribute; for instance, FermionTerm[2] returns the third element of the .terms attribute. The print and repr methods are overloaded to show a convenient human-readable representation. The methods .hermitian_conjugate() (an in-place method) and .hermitian_conjugated() (returns a copy) act on the term $T$ to return $T^\dagger$. Note that this convention of methods that end in "ed" returning copies whereas methods without "ed" being in-place occurs throughout the library. We also show a quick way to initialize identity operators and number operators. See the code documentation for further details.

In [27]:
from fermion_operators import FermionTerm, fermion_identity

n_qubits = 8
example_term = FermionTerm(n_qubits, -3.17, [(3, 1), (7, 0)])
identity_fermion = fermion_identity(n_qubits)
print('Our example term is: {}'.format(example_term))
print('Below are some manipulations of that term.')
print 2. * example_term
print abs(example_term)
print len(example_term)
print example_term[1]
example_term[1] = (2, 0)
example_term *= 5.j
print example_term
print example_term.hermitian_conjugated()
print example_term.is_molecular_term()

print('\nOur identity term is represented as: {}').format(identity_fermion)
identity_fermion *= 3.
print('Is the identity term equal to the other term? FermiLib says: {}'.format(
    identity_fermion == example_term))
print('Are they not equal? FermiLib says: {}'.format(
     identity_fermion != example_term))

print('\nBelow we loop through the ladder operators.')
for ladder_operator in example_term:
    print ('{} is a ladder operator in the example term.'.format(ladder_operator))

Our example term is: -3.17 (3+ 7)
Below are some manipulations of that term.
-6.34 (3+ 7)
3.17 (3+ 7)
2
(7, 0)
(-0-15.85j) (3+ 2)
(-0+15.85j) (2+ 3)
False

Our identity term is represented as: 1.0 ()
Is the identity term equal to the other term? FermiLib says: False
Are they not equal? FermiLib says: True

Below we loop through the ladder operators.
(3, 1) is a ladder operator in the example term.
(2, 0) is a ladder operator in the example term.


## The FermionOperator data structure represents arbitrary fermion operators

The FermionOperator data structure stores arbitrary sums of FermionTerm objects. For instance, the sum $-1.7 \, a^\dagger_3 a_1 + (1 + 2i) \, a^\dagger_4 a^\dagger_3 a_9 a_1 + 0.8\, a_3 a_{11}$ would be an example of a FermionOperator. Like FermionTerm, FermionOperator inherits from a more general parent class which it shares with QubitOperator, LocalOperator, which is found in the module local_operator.py. The FermionOperator data structure is implemented as a wrapper to a python dictionary. Internally, the keys for this dictionary are the .terms attribute of a the FermionTerm object and the values are the .coefficient attribute of the FermionTerm. 

To initialize a FermionOperator one should pass n_qubits, followed by a list of FermionTerms. Alternatively, a sum or difference of FermionTerms automatically returns a FermionOperator. The FermionOperator supports many built-in operands from the python data model such as $+$, $-$, $+=$, $-=$, $*$, $**$, $==$, $!=$, abs, print, iter (aka "in"), len, and del. The reason that FermionOperator is implement as a dictionary is so that terms can be quickly added to the class. Note that one can set or look up the coefficient of a term in a FermionOperator by passing an instance of FermionTerm to the slice operator [] of FermionOperator.

In [30]:
from fermion_operators import FermionOperator, FermionTerm, number_operator
n_qubits = 8
example_term_a = FermionTerm(n_qubits, -3.17, [(3, 1), (7, 0), (6, 1), (7, 1)])
example_term_b = FermionTerm(n_qubits, -88.j, [(1, 1), (2, 0)])
example_term_c = FermionTerm(n_qubits, 2., [(6, 1), (3, 0)])
fermion_number = number_operator(n_qubits)

example_operator = FermionOperator(n_qubits, [example_term_a, example_term_b])
alternative_operator = example_term_a + example_term_b
print('Our example FermionOperators are:\n{}'.format(example_operator))
print('We could also have made this operator by just summing together the terms.\n' +
      'Does FermiLib agree that these methods of initialization are equivalent?\n' +
      str(alternative_operator == example_operator))

print('\nBelow is a number operator:\n{}'.format(fermion_number))

print('Demonstration 1.')
alternative_operator *= -1.
print(alternative_operator)

print('Demonstration 2.')
alternative_operator -= 2. * example_term_c
print(alternative_operator)

print('Demonstration 3.')
squared_operator = example_operator ** 2
print(squared_operator)

print('Demonstration 4.')
print(example_operator[example_term_b])
example_operator[example_term_b] += 1.
print(example_operator[example_term_b])

print('\nDemonstration 5.')
for term in example_operator:
    print('Now looping over term {}.'.format(term))
print(example_term_a in example_operator)
print(example_term_c in example_operator)

print('\nDemonstration 6.')
del example_operator[example_term_a]
print example_operator

Our example FermionOperators are:
-88j (1+ 2)
-3.17 (3+ 7 6+ 7+)

We could also have made this operator by just summing together the terms.
Does FermiLib agree that these methods of initialization are equivalent?
True

Below is a number operator:
1.0 (4+ 4)
1.0 (3+ 3)
1.0 (1+ 1)
1.0 (6+ 6)
1.0 (2+ 2)
1.0 (7+ 7)
1.0 (0+ 0)
1.0 (5+ 5)

Demonstration 1.
88j (1+ 2)
(3.17-0j) (3+ 7 6+ 7+)

Demonstration 2.
88j (1+ 2)
-4.0 (6+ 3)
(3.17-0j) (3+ 7 6+ 7+)

Demonstration 3.
278.96j [(3, 1), (7, 0), (6, 1), (7, 1), (1, 1), (2, 0)]
10.0489 [(3, 1), (7, 0), (6, 1), (7, 1), (3, 1), (7, 0), (6, 1), (7, 1)]
(-7744+0j) [(1, 1), (2, 0), (1, 1), (2, 0)]
278.96j [(1, 1), (2, 0), (3, 1), (7, 0), (6, 1), (7, 1)]

Demonstration 4.
-88j
(1-88j)

Demonstration 5.
Now looping over term (1-88j) (1+ 2).
Now looping over term -3.17 (3+ 7 6+ 7+).
True
False

Demonstration 6.
(1-88j) (1+ 2)



## Normal-ordering
Below, we show off routines for normal-ordering both FermionTerms and FermionOperators. Note that the normal-ordering method on FermionTerm automatically returns a FermionOperator, not a FermionTerm, as in general the normal-ordered form of a FermionTerm is a sum of FermionTerms.

In [38]:
from fermion_operators import FermionOperator, FermionTerm
n_qubits = 8
example_term_a = FermionTerm(n_qubits, -3.17, [(3, 1), (7, 0), (6, 1), (7, 1)])
example_term_b = FermionTerm(n_qubits, -88.j, [(1, 1), (2, 0)])
example_term_c = FermionTerm(n_qubits, 2., [(6, 1), (3, 0)])
example_operator = FermionOperator(n_qubits, [example_term_a, example_term_b])

print('After normal-ordering, term {} becomes operator\n{}'.format(
    example_term_a, example_term_a.normal_ordered()))

print('We can also normal-order operators. For instance, operator\n{}becomes operator\n{}'.format(
    example_operator, example_operator.normal_ordered()))

After normal-ordering, term -3.17 (3+ 7 6+ 7+) becomes operator
(3.17+0j) (7+ 6+ 3+ 7)
(-3.17+0j) (6+ 3+)

We can also normal-order operators. For instance, operator
-88j (1+ 2)
-3.17 (3+ 7 6+ 7+)
becomes operator
-88j (1+ 2)
(3.17+0j) (7+ 6+ 3+ 7)
(-3.17+0j) (6+ 3+)



## Lattice models

FermiLib also supports generating various lattice models of fermions such as the Hubbard model (and others that will be added once papers are out). Below are some examples showing code to generate Hubbard models.

In [71]:
from hubbard import fermi_hubbard
x_dimension = 3
y_dimension = 2
tunneling = 2.
coulomb = 1.
magnetic_field = 0.5
chemical_potential = 0.25
periodic = 1
spinless = 0
hubbard_model = fermi_hubbard(
    x_dimension, y_dimension, tunneling, coulomb, chemical_potential, magnetic_field, periodic, spinless)
print(hubbard_model)

1.0 (10+ 10 11+ 11)
-2.0 (0+ 6)
-0.75 (10+ 10)
-4.0 (10+ 8)
-2.0 (11+ 5)
1.0 (8+ 8 9+ 9)
-2.0 (8+ 2)
-4.0 (5+ 3)
0.25 (1+ 1)
-2.0 (1+ 7)
-2.0 (9+ 3)
-0.75 (2+ 2)
-4.0 (2+ 4)
-4.0 (9+ 11)
1.0 (4+ 4 5+ 5)
0.25 (5+ 5)
0.25 (11+ 11)
1.0 (0+ 0 1+ 1)
-2.0 (2+ 0)
-0.75 (0+ 0)
-2.0 (6+ 0)
-2.0 (7+ 9)
-4.0 (8+ 10)
-2.0 (5+ 11)
-2.0 (1+ 3)
0.25 (3+ 3)
-2.0 (3+ 9)
1.0 (2+ 2 3+ 3)
-2.0 (9+ 7)
-2.0 (8+ 6)
0.25 (7+ 7)
-4.0 (11+ 9)
-2.0 (6+ 8)
-2.0 (10+ 4)
-2.0 (2+ 8)
-0.75 (4+ 4)
-4.0 (4+ 2)
-2.0 (7+ 1)
-2.0 (4+ 10)
-0.75 (6+ 6)
-4.0 (3+ 5)
-2.0 (0+ 2)
-2.0 (3+ 1)
1.0 (6+ 6 7+ 7)
0.25 (9+ 9)
-0.75 (8+ 8)



## QubitOperators and QubitTerms data structures

Also core to FermiLib's functionality is the representation of local qubit Hamiltonians. The QubitTerm and QubitOperator data structures share much in common with FermionTerm and FermionOperator data structures, which is why they inherit from the same parent classes in local_terms.py and local_operators.py.

An example of a QubitTerm is the local Hamiltonian $0.8 \, X_1 Y_7 Z_{12}$ where $X_1$, $Y_7$, and $Z_{12}$ are Pauli operators acting on tensor factors 1, 7 and 12. A key difference between FermionTerm and QubitTerm is that the .terms attribute of QubitTerm differs from the .terms attribute of FermionTerm. Specifically, instead of $a^\dagger$ and $a$ operators denoted by 1 and 0, QubitTerms encode $X$, $Y$ and $Z$ using those strings. For instance, for the term mentioned above, the .terms attribute would be [(1, 'X'), (7, 'Y'), (12, 'Z')]. Note that while the .terms attribute of FermionTerm is not necessarily sorted by orbital index, the .term attribute of QubitTerm is always sorted by tensor factor. This allows for faster term multiplication.

An example of QubitOperator is a sum of local Hamiltonians; e.g., $0.8 \, X_1 Y_7 Z_{12} + 3.17 \, Z_2 Z_3$. In pretty much all regards, the QubitOperator class supports the same operations and access of the FermionOperator class. Note that the identity term is again represented by an empty terms list and an empty QubitOperator terms list corresponds to the zero operator. We demonstrate some of these functionalities below.

In [61]:
from qubit_operators import QubitTerm, QubitOperator, qubit_identity

n_qubits = 8
example_term_a = QubitTerm(n_qubits, 0.3, [(1, 'Y'), (4, 'X'), (2, 'X'), (6, 'Z')])
example_term_b = QubitTerm(n_qubits, 17, [(0, 'X'), (3, 'X')])
example_term_c = 8.9 * qubit_identity(n_qubits)
example_operator = example_term_a - example_term_b
alternative_operator = example_term_a + example_term_c
print('The example operators is:\n{}'.format(example_operator))

print('Below are some examples of manipulating it.')
example_operator *= alternative_operator
print(example_operator)

print(len(example_operator))
print(example_operator[example_term_c])
print(example_operator == alternative_operator)


The example operators is:
-17.0 X0 X3
0.3 Y1 X2 X4 Z6

Below are some examples of manipulating it.
-151.3 X0 X3
0.09 I
-5.1 X0 Y1 X2 X3 X4 Z6
2.67 Y1 X2 X4 Z6

4
0.09
False


## Forward and reverse Jordan-Wigner transformations

One can easily map back and forth between FermionTerms/Operators and QubitTerms/Operators using .jordan_wigner_transform(), which acts on fermions, and .reverse_jordan_wigner(), which acts on qubits. Note that like all methods that output an instance of a different class type, these are not in-place methods. We are working with James Whitfield and Vojtech Havlicek to incorporate Bravyi-Kitaev transforms and other, more general methods.

In [70]:
from fermion_operators import FermionOperator, FermionTerm
n_qubits = 8
example_term = FermionTerm(n_qubits, -3.17, [(3, 1), (7, 0), (6, 1), (7, 1)])

print('The example terms is\n{}\nwhich becomes \n{}under the Jordan-Wigner transform.'.format(
    example_term, example_term.jordan_wigner_transform()))

print('\nWe can also reverse transform qubit operators. This has many uses.')
for qubit_term in example_term.jordan_wigner_transform():
    reversed_term = qubit_term.reverse_jordan_wigner()
    reversed_term.normal_order()
    print('The reverse Jordan-Wigner transform of {} is:\n{}\n'.format(
        qubit_term, reversed_term))

The example terms is
-3.17 (3+ 7 6+ 7+)
which becomes 
-0.39625j X3 Z4 Z5 Y6 Z7
(0.39625+0j) X3 Z4 Z5 X6 Z7
(-0.39625+0j) Y3 Z4 Z5 Y6
-0.39625j Y3 Z4 Z5 X6
(0.39625+0j) X3 Z4 Z5 X6
-0.39625j X3 Z4 Z5 Y6
-0.39625j Y3 Z4 Z5 X6 Z7
(-0.39625+0j) Y3 Z4 Z5 Y6 Z7
under the Jordan-Wigner transform.

We can also reverse transform qubit operators. This has many uses.
The reverse Jordan-Wigner transform of -0.39625j X3 Z4 Z5 Y6 Z7 is:
(0.39625+0j) (6+ 3)
(-0.39625+0j) (6+ 3+)
(0.7925+0j) (7+ 7 6 3)
(0.7925+0j) (7+ 6+ 3+ 7)
(0.7925+0j) (7+ 6+ 7 3)
(-0.39625+0j) (6 3)
(-0.7925+0j) (7+ 3+ 7 6)
(-0.39625+0j) (3+ 6)


The reverse Jordan-Wigner transform of (0.39625+0j) X3 Z4 Z5 X6 Z7 is:
(0.39625+0j) (6+ 3)
(-0.39625+0j) (6+ 3+)
(-0.7925+0j) (7+ 7 6 3)
(0.7925+0j) (7+ 6+ 3+ 7)
(0.7925+0j) (7+ 6+ 7 3)
(0.39625+0j) (6 3)
(0.7925+0j) (7+ 3+ 7 6)
(0.39625+0j) (3+ 6)


The reverse Jordan-Wigner transform of (-0.39625+0j) Y3 Z4 Z5 Y6 is:
(-0.39625+0j) (6+ 3)
(-0.39625+0j) (6+ 3+)
(0.39625+0j) (6 3)
(-0.3962

## Hubbard model mapped to qubits

It should be pretty obvious how to do this now.

In [74]:
from hubbard import fermi_hubbard
x_dimension = 3
y_dimension = 2
tunneling = 2.
coulomb = 1.
magnetic_field = 0.5
chemical_potential = 0.25
periodic = 1
spinless = 0
hubbard_model = fermi_hubbard(
    x_dimension, y_dimension, tunneling, coulomb, chemical_potential, magnetic_field, periodic, spinless)
qubit_hamiltonian = hubbard_model.jordan_wigner_transform()
print(qubit_hamiltonian)
print('Model has {} terms post-Jordan-Wigner transform.'.format(len(qubit_hamiltonian)))

(-1+0j) Y6 Z7 Y8
(-1+0j) X0 Z1 Z2 Z3 Z4 Z5 X6
(-1+0j) X4 Z5 Z6 Z7 Z8 Z9 X10
(0.25+0j) Z0 Z1
(-0.375+0j) Z7
(-2+0j) Y3 Z4 Y5
(-1+0j) Y0 Z1 Z2 Z3 Z4 Z5 Y6
(-1+0j) Y1 Z2 Z3 Z4 Z5 Z6 Y7
(0.125+0j) Z4
(-1+0j) Y2 Z3 Z4 Z5 Z6 Z7 Y8
(0.125+0j) Z10
(-2+0j) Y8 Z9 Y10
(-1+0j) Y5 Z6 Z7 Z8 Z9 Z10 Y11
(0.25+0j) Z10 Z11
(0.125+0j) Z0
(0.125+0j) Z2
(-0.375+0j) Z5
(-1+0j) X5 Z6 Z7 Z8 Z9 Z10 X11
(-0.375+0j) Z1
(-2+0j) X9 Z10 X11
(-1+0j) Y0 Z1 Y2
(-1+0j) Y7 Z8 Y9
(-1+0j) X3 Z4 Z5 Z6 Z7 Z8 X9
(0.25+0j) Z6 Z7
(0.125+0j) Z8
(-2+0j) X3 Z4 X5
(-1+0j) X0 Z1 X2
(0.25+0j) Z2 Z3
(-0.375+0j) Z3
(-0.375+0j) Z9
(-1+0j) X2 Z3 Z4 Z5 Z6 Z7 X8
(-1+0j) Y4 Z5 Z6 Z7 Z8 Z9 Y10
(-1+0j) Y1 Z2 Y3
(0.25+0j) Z4 Z5
(-2+0j) X8 Z9 X10
(-2+0j) Y9 Z10 Y11
(-0.375+0j) Z11
(0.125+0j) Z6
(-1+0j) X6 Z7 X8
(-1+0j) Y3 Z4 Z5 Z6 Z7 Z8 Y9
(0.25+0j) Z8 Z9
(-2+0j) X2 Z3 X4
(-2+0j) Y2 Z3 Y4
(-1+0j) X1 Z2 X3
(-1+0j) X7 Z8 X9
(-1+0j) X1 Z2 Z3 Z4 Z5 Z6 X7

Model has 46 terms post-Jordan-Wigner transform.


## Psi4, MolecularData class and MolecularOperator data structure
blah

In [100]:
from molecular_data import MolecularData
from run_psi4 import run_psi4

# Set parameters.
geometry = [('Li', (0., 0., 0.)), ('H', (0., 0., 0.7414))]
basis = 'sto-3g'
multiplicity = 1
charge = 0
description = 'eq'
active_space_start = 1
active_space_stop = 3
run_scf = 1
run_mp2 = 0
run_cisd = 1
run_ccsd = 0
run_fci = 0

# Generate instance of MolecularData.
molecule = MolecularData(geometry, basis, multiplicity, charge, description)
print('Molecule has automatically generated name {}'.format(molecule.name))
print('This molecule has {} atoms and {} electrons.'.format(
    molecule.n_atoms, molecule.n_electrons))
print('Without active space, calculation involves {} orbitals and {} qubits.'.format(
    molecule.n_orbitals, molecule.n_qubits))

# Run Psi4.
molecule = run_psi4(molecule,
                    run_scf=run_scf,
                    run_mp2=run_mp2,
                    run_cisd=run_cisd,
                    run_ccsd=run_ccsd,
                    run_fci=run_fci)

# Print out some interesting facts.
print('Hartree-Fock energy is {}.'.format(molecule.hf_energy))

# Get the Hamiltonian.
molecular_hamiltonian = molecule.get_molecular_hamiltonian(
    active_space_start, active_space_stop)
fermion_hamiltonian = molecular_hamiltonian.get_fermion_operator()
qubit_hamiltonian = molecular_hamiltonian.jordan_wigner_transform()

print('\nThe associated fermion Hamiltonian follows:\n{}'.format(fermion_hamiltonian))
print('The associated qubit Hamiltonian follows:\n{}'.format(qubit_hamiltonian))


Molecule has automatically generated name H1-Li1_sto-3g_singlet_eq
This molecule has 2 atoms and 4 electrons.
Without active space, calculation involves 6 orbitals and 12 qubits.
Hartree-Fock energy is -7.54543993698.

The associated fermion Hamiltonian follows:
-0.0161022352963 (0+ 1+ 3 0)
0.253875333403 (1+ 0+ 0 1)
-0.0161022352963 (1+ 0+ 0 3)
0.128645844394 (2+ 0+ 0 2)
0.128645844394 (3+ 1+ 1 3)
0.0046615465742 (3+ 2+ 0 1)
0.128645844394 (0+ 3+ 3 0)
-0.0035639754243 (2+ 1+ 3 2)
-0.0161022352963 (2+ 1+ 1 0)
-0.80917095887 (1+ 1)
-0.0035639754243 (0+ 3+ 3 2)
0.167969536706 (3+ 2+ 2 3)
0.128645844394 (1+ 2+ 2 1)
-0.442749043032 (2+ 2)
-0.0035639754243 (1+ 2+ 2 3)
0.128645844394 (1+ 3+ 3 1)
0.253875333403 (0+ 1+ 1 0)
0.0322049222066 (3+ 1)
0.167969536706 (2+ 3+ 3 2)
-0.0161022352963 (3+ 0+ 0 1)
-0.0035639754243 (2+ 3+ 1 2)
0.0046615465742 (2+ 0+ 2 0)
0.0322049222066 (2+ 0)
0.0046615465742 (3+ 0+ 2 1)
-0.80917095887 (0+ 0)
0.0046615465742 (0+ 2+ 0 2)
0.0046615465742 (2+ 1+ 3 0)
0.0046615

## MolecularOperator data structure compactly represents molecular Hamiltonians and RDMs

blah